In [17]:
#A1
import numpy as np

def doExpectation(A,B,data,n):
    hiddenMatrix=[[],[]]
    for datapoint in data:
        hiddenMatrix[0].append(verantworlichkeitA(A, B, datapoint, n ))
    for datapoint in data:
        hiddenMatrix[1].append(verantworlichkeitB(A, B, datapoint, n ))
    return hiddenMatrix

def pr(theta,datapoint,n):
    return (theta**(n*datapoint))*((1-theta)**(n*(1-datapoint)))

def verantworlichkeitA(A, B, datapoint, n ):
    return pr(A,datapoint,n)/((pr(A,datapoint,n)+pr(B,datapoint,n)))
                              
def verantworlichkeitB(A, B, datapoint, n ):
    return pr(B,datapoint,n)/(pr(B,datapoint,n)+pr(A,datapoint,n))
                              
data=[0.4,0.9,0.8,0.3,0.7]
A=0.4
B=0.8
n=5
hiddenMatrix=doExpectation(A,B,data,n)
print(hiddenMatrix[0])
print(hiddenMatrix[1])

[0.8709677419354839, 0.07110380330949087, 0.15789473684210525, 0.9429681293665482, 0.31473023590887406]
[0.12903225806451607, 0.9288961966905092, 0.8421052631578948, 0.05703187063345183, 0.6852697640911258]


A2)
Die Abbruchbedingung ist das Überprüfen der Theta Werte.Falls sich diese nicht um 0.01 verändern wird die Iteration abgebrochen.


In [98]:
#A3 HIER WIRD DER LLOYD-ALLGORITHMUS AUS DER MUSTERLOESUNG VERWENDET

import random, re
import numpy as np
import matplotlib.pyplot as plt
import math

def read_data(filename):
    """Read in data file and return list containing
       tuples for each gene and its expression vector"""
    gene_expvec = []
    with open(filename) as fin:
        for zeile in fin:
            zeile = zeile.rstrip()
            if not zeile.startswith("#"):
                l = zeile.split("\t")
                gene = l[0]
                
                # create numpy array for expression vector:
                coords = []
                for coord in l[1:]:
                    coords.append(float(coord))
                expvec = np.array(coords)
                
                # append tuple to gene expression list:
                gene_expvec.append((gene,expvec))
    return gene_expvec

def d_euklidean(datapoint1,datapoint2):
    """Calculate Euklidean distance for two datapoints"""
    """(Datapoints are given as tuple(gene,expvec))   """
    expvec1 = datapoint1[1]
    expvec2 = datapoint2[1]
    
    # calculate the square root of summed squared differences at each coordinate 
    dist = np.sqrt(np.sum(np.square(expvec1 - expvec2)))
    return dist


def d_DataPoint_Centers(datapoint,centers):
    # for each center, calculate distance from the datapoint:
    distances = [(d_euklidean(datapoint,center), datapoint, center) for center in centers]
    # return minimum distance and respective datapoint and center:
    mindist = min(distances, key = lambda x: x[0])
    return mindist

# This program uses function definitions from Aufgabe 1

def doExpectation(centers,datapoints):
    hiddenMatrix=[[],[],[],[],[],[]]
    k=0
    for center in centers:
        for datapoint in datapoints:
            hiddenMatrix[k].append(formelPhysik(centers,center,datapoint,1))
            
        k=k+1        
    return hiddenMatrix

def formelPhysik(centers,center,datapoint,beta):
    nenner=0
    zaehler=math.e**(-beta*d_euklidean(datapoint,center))
    for center in centers:
        nenner = nenner + math.e**-beta*d_euklidean(datapoint,center)
    #print(zaehler/nenner)
    return zaehler/nenner

def centers_equal(centers1,centers2):
    """Check, if two centers are equal"""
    equal = True    
    for i in range(len(centers1)):
        name1, expvec1 = centers1[i]
        name2, expvec2 = centers2[i]
        
        if np.any(expvec1 != expvec2):
            equal = False
    
    return equal


def center_of_gravity(distances,center):
    """For all datapoints in a cluster, defined by its center
       calculate the center of gravity expression vector"""
    center_name, center_expvec = center
    
    m = len(center_expvec)
    center_of_grav_expvec = np.zeros(m)
    
    n = 0
    # distances is list of tuples (distance, datapoint, center)
    for distancei, datapointi, centeri in distances:
        centeri_name, centeri_expvec = centeri
        
        # check if datapoint belongs to the cluster, that we are looking at:
        if centeri_name == center_name:
            datapoint_name, datapoint_expvec = datapointi
            center_of_grav_expvec += datapoint_expvec
            n += 1
   
    center_of_grav_expvec /= n
    return center_of_grav_expvec


# get number of clusters:
k = 6
filename = "data.txt"

# Each datapoint is represented a tuple: (gene, coordinates)
# datapoints is a list of tuples:
datapoints = read_data(filename)

# randomly select k datapoints as centers:
centers = random.sample(datapoints,k)

# set center names:
for i in range(k):
    name, expvec = centers[i]
    centers[i] = (str(i), expvec)

# do k-means:
converged = False
while not converged:
    # centers to clusters:
    distances = [d_DataPoint_Centers(datapoint, centers) for datapoint in datapoints]
    hiddenMatrix=doExpectation(centers,datapoints)#HIER IST DIE ERSTELLUNG DER HIDDENMATRIZEN je nach Centeraenderung
    # distances is list of tuples (distance, datapoint, center)
    
    # make a copy of centers:
    centers_cp = centers.copy()

    # clusters to centers:
    for i in range(len(centers)):
        expvec = center_of_gravity(distances,centers[i])
        centers[i] = (str(i), expvec)
        
    if centers_equal(centers_cp,centers):
        converged = True

#Zuteilung der DataPoints zu den Clustern
cluster2datapoints = [[],[],[],[],[],[]]
for i in range(0,231):
    currentmaximum=0
    for j in range(0,6):
        if hiddenMatrix[j][i] > currentmaximum:
            currentmaximum=hiddenMatrix[j][i]
            maxj=j
    #print(maxj)
    cluster2datapoints[maxj].append(hiddenMatrix[maxj][i])
#printen der HM.txt
with open('HM.txt', 'w') as testfile:
    for row in hiddenMatrix:
        testfile.write(' '.join([str(a) for a in row])+'\n')
        testfile.write('\n')

In [95]:
#A4
print (len(cluster2datapoints[0]))
print (len(cluster2datapoints[1]))
print (len(cluster2datapoints[2]))
print (len(cluster2datapoints[3]))
print (len(cluster2datapoints[4]))
print (len(cluster2datapoints[5]))

102
38
40
24
9
18
